# Einführung in Unsupervised Learning: k-Means Clustering mit Python

In diesem Übungsbeispiel wirst du Unsupervised Learning ausprobieren, bei der Algorithmen Muster und Strukturen in Datensätzen finden, ohne dabei auf vorherige Labels oder Anweisungen angewiesen zu sein. Ein weit verbreiteter und intuitiver Ansatz innerhalb dieser Kategorie ist das Clustering, und der k-Means Algorithmus ist eine der bekanntesten Techniken dafür.

Clustering hilft uns, Einblicke in die Struktur unserer Daten zu gewinnen, indem es ähnliche Datenpunkte in Gruppen (Cluster) zusammenfasst. Diese Technik findet breite Anwendung in verschiedenen Bereichen, darunter Marktforschung, Bilderkennung und mehr, um verborgene Muster innerhalb des Datensatzes zu identifizieren.

In diesem Tutorial werden wir:

    Einen synthetischen Datensatz mit scikit-learn generieren, der sich gut für eine Clusteranalyse eignet.
    Den k-Means Algorithmus anwenden, um unsere Daten in Cluster zu unterteilen.
    Die Ergebnisse visualisieren, um ein Verständnis dafür zu entwickeln, wie k-Means die Daten organisiert und welche Einsichten wir daraus ziehen können.

Ziel dieser Übung ist es, ein grundlegendes Verständnis für Clustering und seine Anwendung mit dem k-Means Algorithmus in Python zu entwickeln.

Die Codezellen können schrittweise ausgeführt werden und es kann mit den Eingabeparamtern für das K-Means-Modell experimentiert werden.


Danach kann ein eigenes Bild verwendet werden, um dieses zu segmentieren. Auch hier kann mit den Eingabeparamteren experimentiert werden.

In [ ]:
# run the cell
# Import der nötigen Bibliotheken
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs



# Schritt 1: Wir generieren zufällige Daten
n_samples = 600
random_state = 40
X, _ = make_blobs(n_samples=n_samples, centers=6, cluster_std=0.60, random_state=random_state)



In der folgenden Zelle kann mit der Anzahl der Cluster experimentiert werden. Dafür muss der Parameter `n_clusters` geändert werden.

In [ ]:
# hier kann n_clusters geändert werden
# Schritt 2: Anwendung des k-Means Algorithmus
kmeans = KMeans(n_clusters=6, random_state=random_state)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)


Im nächsten Schritt visualisieren wir die Ergebnisse des k-Means Clustering. Wie sollten wir `n_clusters` wählen, um sicherzustellen, dass die Anzahl der Cluster die Verteilung der Datenpunkte im Bild sinnvoll widerspiegelt?  

**Aufgabe:**  

Experimentiere mit unterschiedlichen Werten für `n_clusters`. Überlege, was ein „sinnvolles“ Clustering in diesem Kontext bedeutet und wie man entscheiden kann, wann die Clusteranzahl „richtig“ ist.

In [ ]:
# run the cell
# Schritt 3: Visualisierung der Cluster
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5)
plt.title("k-Means Clustering")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

## Bildsegmentierung mit K-Means


In diesem Beispiel verwenden wir k-Means, um die Farben im Bild zu clustern, was zu einer reduzierten Farbpalette führt. Jedes Pixel wird dann durch die nächste Farbe in dieser Palette ersetzt, was zu einer Art "gemaltes" Bild führt, das immer noch das ursprüngliche Bild erkennen lässt, aber nur mit wenigen Farben.

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.cluster import KMeans
from sklearn.utils import shuffle
import imageio.v2 as imageio
from imageio.v2 import imread

Hier kann ein eigenes Bild geladen geben. Dafür den Pfad in `imread()` angeben, oder ein Bild in diesen Ordner hochladen und den Namen des Bildes angeben.

**Aufgabe:**  

Wähle ein eigenes Bild für die Segmentierung und gib den Pfad an.

In [ ]:
# Pfad anpassen
# Schritt 1: Laden eines eigenen Bildes
image = imread('Beispielbild.png') 
image = np.array(image, dtype=np.float64) / 255

Für das Bildsegmentierungsbeispiel mit k-Means nehmen wir nur eine kleinen Teil als Stichprobe des Bildes zum Trainieren des Modells, weil k-Means auf der gesamten Menge der Pixel trainieren zu können, sehr rechenintensiv sein kann. Ein digitales Bild kann sehr sehr viele Pixel enthalten, und der k-Means Algorithmus arbeitet iterativ. Das bedeutet, dass jeder Durchlauf über die gesamten Daten iteriert. Dies kann auf Vollbilddaten sehr zeit- und ressourcenaufwändig sein.

Durch die Reduzierung der Datenmenge auf eine kleine, repräsentative Stichprobe (1%) lassen sich die  wesentlichen Farben des Bildes erfassen und die Rechenzeit reduzieren. Die Annahme hier ist, dass eine zufällig ausgewählte kleine Menge von Pixeln eine gute Annäherung an die gesamte Farbverteilung im Bild bietet. Nach dem Training können die gefundenen Clusterzentren (die repräsentativen Farben) verwendet werden, um das gesamte Bild zu segmentieren.

**Aufgabe:**  

Experimentiere mit der Anzahl von `n_clusters`!

In [ ]:
# run the cell
# Schritt 2: Reshape das Bild zu einer 2D-Array von Pixeln und 3 Farbwerten (RGB)
w, h, d = original_shape = tuple(image.shape)
assert d == 3
image_array = np.reshape(image, (w * h, d))

# Schritt 3: Das Modell wird auf einen kleinen Teil der Daten gefittet, dafür reichen etwa 1% des Bildes. 
# Das Spart Zeit und Rechenressourcen
image_sample = shuffle(image_array, random_state=0)[:int(0.01 * w * h)]

kmeans = KMeans(n_clusters=15, random_state=0).fit(image_sample)

Im nächsten Schritt nutzen wr das trainierte Modell und wenden es auf das gesammte Bild an.
Dann werden die Ergebnisse visualisert. Das originalbild, sowie zwei Versionen des Clusterergebnisses.


In [ ]:


# Schritt 4: Vorhersage der Clusterzuordnung für jeden Pixel im Bild
labels = kmeans.predict(image_array)

##### Visualisierung
# Funktion zum Erstellen eines segmentierten Bildes aus den Labels
def recreate_image(codebook, labels, w, h):
    """Recreate the (compressed) image from the code book & labels"""
    d = codebook.shape[1]
    image = np.zeros((w, h, d))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = codebook[labels[label_idx]]
            label_idx += 1
    return image



# Zusätzliche Schritt: Zuweisen zufälliger Farben zu jedem Cluster
np.random.seed(0)  # Für reproduzierbare Farben
colors = np.random.rand(kmeans.n_clusters, 3)

# Funktion zum Erstellen des Bildes, wo jeder Cluster eine zufällige Farbe hat
def recreate_image_random_colors(codebook, labels, w, h):
    """Recreate the image from the code book & labels with random colors"""
    d = codebook.shape[1]
    image = np.zeros((w, h, d))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = colors[labels[label_idx]]
            label_idx += 1
    return image

# Anzeigen aller drei Bilder: Original, Segmentiert mit realen Farben, Segmentiert mit zufälligen Farben
fig, ax = plt.subplots(1, 3, figsize=(18, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))


ax[0].imshow(image)
ax[0].set_title('Original Image')
ax[1].imshow(recreate_image(kmeans.cluster_centers_, labels, w, h))
ax[1].set_title('Segmented Image (K-Means)')
ax[2].imshow(recreate_image_random_colors(kmeans.cluster_centers_, labels, w, h))
ax[2].set_title('Segmented Image with Random Cluster Colors')
plt.show()
